In [41]:
import review_RTP as RTP
import numpy as np
import geopandas as gpd
import os, re
import pandas as pd
import glob

In [2]:
df = RTP.getCombinedTables(cat='added', export=True)

In [3]:
# new projects?
df

,Name,GeographicLimits,Description,PrimaryJurisdiction,AirQualityStatus,EstimatedCost,EstimatedYearofConstruction,YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP,FunctionalClass,JurisdictionalProject#,Category,In
0,Eugene-Springfield Highway (OR 126),@ Mohawk Boulevard Interchange,Add lanes on ramps,non exempt; regionally significant,ODOT,2000000.0,2030-2034,3066559.0,3464862.0,0.68,821,Other Freeways and Expressways,,Added Freeway Lanes or Major Interchange Impro...,AutoConstrained
1,Henderson Avenue,Franklin Boulevard to East 19th Avenue,Modify Henderson Avenue with a 3-lane cross-se...,"Springfield, Lane County",Exempt - Safety- widen lanes/ resurfacing/ add...,3400000.0,2035-2040,NaN,NaN,0.39,827,Local (current),"STSP-R-17, LCTSP-3",Urban Standards,AutoConstrained
2,East 19th Avenue,Henderson Avenue to McVay Hwy,Change East 19th Avenue to a 3-lane cross-sect...,Springfield,Exempt - Safety- widen lanes/ resurfacing/ add...,3500000.0,2030-2034,NaN,NaN,0.49,828,Minor Collector,R-18,Urban Standards,AutoConstrained
3,Yolanda Avenue,23rd Street to 31st Street,Modify Yolanda Avenue to a 2-lane cross-sectio...,Springfield,Exempt - Safety- widen lanes/ resurfacing; Air...,460000.0,2025-2029,NaN,NaN,NaN,784,Minor Collector,R-26,Urban Standards,AutoConstrained
4,Horn Lane,Lake Drive to River Road,Bicycle Boulevard,Eugene,exempt - Air Quality - bike and ped facilities,116000.0,2020-2024,NaN,NaN,0.75,521,Minor Collector,PB-161,On-Street Lanes or Routes Without Road Project,BikeConstrained
5,By Gully Path Extension,Pioneer Parkway to 5th Street,Construct a new multi-use 12-foot wide path fr...,"Willamalane, Springfield",exempt - Air Quality - bike and ped facilities,-,Beyond 20 years,NaN,NaN,0.11,812,…,PB-11,Multi-Use Paths Without Road Project,BikeIllustrative


In [4]:
df.shape

(6, 15)

In [4]:
targetLayers = RTP.targetLayers()

In [5]:
targetLayers

['Constrained_Roadway_lines',
 'Constrained_Roadway_points',
 'Illustrative_Roadway_lines',
 'Illustrative_Roadway_points',
 'Constrained_BikePed',
 'Constrained_BikePed_points',
 'Illustrative_BikePed']

In [2]:
inpath = r'T:\MPO\RTP\FY16 2040 Update\Data\RTP_2040_Data.gdb'
path = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Data'

In [3]:
tablePatterns = np.array(['Auto Constrained', 'Auto Illustrative', 'Bike Constrained', 'Bike Illustrative'])
layerPatterns = np.array(['Constrained_Roadway', 'Illustrative_Roadway', 'Constrained_BikePed', 'Illustrative_BikePed'])

In [8]:
for layer in targetLayers:
    print(layer)
    gdf = gpd.read_file(inpath, layer=layer)
    l = layer.split('_')
    layerPattern = l[0] + '_' + l[1] 
    i=np.min(np.where(layerPatterns == layerPattern))
    tablePattern = tablePatterns[i]
    res=RTP.getIDs(Tablepattern=tablePattern,
           Layerpattern=layerPattern)
    commonIDs=RTP.matchID(res[0], res[1])[2]
    newgdf = gdf[gdf.RTP_ID.isin(commonIDs)]
    newgdf.to_file(os.path.join(path, layer+'.shp'))

Constrained_Roadway_lines


<ipython-input-8-e935b767af78>:12: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  newgdf.to_file(os.path.join(path, layer+'.shp'))


Constrained_Roadway_points
Illustrative_Roadway_lines
Illustrative_Roadway_points
Constrained_BikePed
Constrained_BikePed_points
Illustrative_BikePed


In [6]:
for layer in targetLayers:
    print(layer)
    gdf = gpd.read_file(os.path.join(path, layer+'.shp'))
    l = layer.split('_')
    layerPattern = l[0] + '_' + l[1]
    i=np.min(np.where(layerPatterns == layerPattern))
    tablePattern = tablePatterns[i]  
    df = RTP.getCombinedTables(cat='common', export=True, byCategory=True, category=tablePattern.replace(' ', ''))[1]
    RTP.shortenColnames(df)
    newgdf = gdf[['RTP_ID', 'geometry']].merge(df, on='RTP_ID')
    newgdf.to_file(os.path.join(path, 'GISData', layer+'.shp'))

Constrained_Roadway_lines
Constrained_Roadway_points
Illustrative_Roadway_lines
Illustrative_Roadway_points
Constrained_BikePed
Constrained_BikePed_points
Illustrative_BikePed


In [7]:
df = RTP.modifyRTP(RTP.combineTables())

In [5]:
df45 = RTP.modifyRTP(RTP.combineTables(year=2045))

In [6]:
df45.head()

,Name,GeographicLimits,Description,PrimaryJurisdiction,EstimatedCost,EstimatedYearofConstruction,YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP,Category
0,Beltline Local Arterial Bridge,Beaver Street to Delta Highway,Construct new 2-lane arterial bridge over the ...,"ODOT, Lane County, City of Eugene",83000000.0,2025-2029,NaN,NaN,0.95,512,New Arterial Link or Interchange
1,Eugene-Springfield Highway (SR-126),at Main Street,Construct interchange,ODOT,50000000.0,2030-2034,NaN,NaN,0.0,27,New Arterial Link or Interchange
2,Eugene-Springfield Highway (SR-126),at 52nd Street,Construct interchange,ODOT,40000000.0,2025-2029,NaN,NaN,0.0,30,New Arterial Link or Interchange
3,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Belltine Highway and provide inte...,"ODOT, Eugene",28100000.0,2030-2034,NaN,NaN,1.1,312,Added Freeway Lanes or Major Interchange Impro...
4,Delta/Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,20000000.0,2020-2024,NaN,NaN,0.25,638,Added Freeway Lanes or Major Interchange Impro...


In [7]:
len([N for N in list(df.RTP.unique()) if N in list(df45.RTP.unique())])

224

In [8]:
len([N for N in list(df.RTP.unique()) if N not in list(df45.RTP.unique())])

30

In [9]:
[N for N in list(df45.RTP.unique()) if N not in list(df.RTP.unique())]

[]

In [7]:
# check the six added projects in GIS data to see if they are in different categories
reviewPath = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\ProjectReview'

In [8]:
addedProjects = pd.read_csv(os.path.join(reviewPath, 'addedProjects.csv'))

In [35]:
addedProjects

,Name,GeographicLimits,Description,PrimaryJurisdiction,AirQualityStatus,EstimatedCost,EstimatedYearofConstruction,YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP,FunctionalClass,JurisdictionalProject#,Category,In
0,Eugene-Springfield Highway (OR 126),@ Mohawk Boulevard Interchange,Add lanes on ramps,non exempt; regionally significant,ODOT,2000000.0,2030-2034,3066559.0,3464862.0,0.68,821,Other Freeways and Expressways,,Added Freeway Lanes or Major Interchange Impro...,AutoConstrained
1,Henderson Avenue,Franklin Boulevard to East 19th Avenue,Modify Henderson Avenue with a 3-lane cross-se...,"Springfield, Lane County",Exempt - Safety- widen lanes/ resurfacing/ add...,3400000.0,2035-2040,NaN,NaN,0.39,827,Local (current),"STSP-R-17, LCTSP-3",Urban Standards,AutoConstrained
2,East 19th Avenue,Henderson Avenue to McVay Hwy,Change East 19th Avenue to a 3-lane cross-sect...,Springfield,Exempt - Safety- widen lanes/ resurfacing/ add...,3500000.0,2030-2034,NaN,NaN,0.49,828,Minor Collector,R-18,Urban Standards,AutoConstrained
3,Yolanda Avenue,23rd Street to 31st Street,Modify Yolanda Avenue to a 2-lane cross-sectio...,Springfield,Exempt - Safety- widen lanes/ resurfacing; Air...,460000.0,2025-2029,NaN,NaN,NaN,784,Minor Collector,R-26,Urban Standards,AutoConstrained
4,Horn Lane,Lake Drive to River Road,Bicycle Boulevard,Eugene,exempt - Air Quality - bike and ped facilities,116000.0,2020-2024,NaN,NaN,0.75,521,Minor Collector,PB-161,On-Street Lanes or Routes Without Road Project,BikeConstrained
5,By Gully Path Extension,Pioneer Parkway to 5th Street,Construct a new multi-use 12-foot wide path fr...,"Willamalane, Springfield",exempt - Air Quality - bike and ped facilities,-,Beyond 20 years,NaN,NaN,0.11,812,…,PB-11,Multi-Use Paths Without Road Project,BikeIllustrative


In [13]:
newPath = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Data\GISData'

In [34]:
# check if these added/duplicated projects are in the existing GIS data
for ID in addedProjects.RTP:
    tablePattern = re.sub(r"(\w)([A-Z])", r"\1 \2", addedProjects[addedProjects.RTP == ID]['In'].values[0])
    i=np.min(np.where(tablePatterns == tablePattern))
    layerPattern = layerPatterns[i]
    layers = [layer for layer in targetLayers if re.search(r"^{0}".format(layerPattern), layer)]
    for layer in layers:
        gdf = gpd.read_file(inpath, layer=layer)
        if ID in gdf.RTP_ID.values:
            print("Project ID {0} is in the layer {1}".format(ID, layer))
            df = addedProjects[addedProjects.RTP == ID]
            RTP.shortenColnames(df)
            added_gdf = gdf[gdf.RTP_ID == ID][['RTP_ID', 'geometry']].merge(df, on='RTP_ID')
            newgdf = gpd.read_file(newPath, layer=layer)
            newgdf.drop_duplicates(inplace=True, ignore_index=True)
            commonCols = [col for col in newgdf.columns if col in added_gdf.columns]
            updatedgdf = newgdf[commonCols].append(added_gdf[commonCols])
            updatedgdf.to_file(os.path.join(newPath, layer+'.shp'))
        else:
            print("Project ID {0} is NOT in the layer {1}".format(ID, layer))

Project ID 821 is NOT in the layer Constrained_Roadway_lines
Project ID 821 is in the layer Constrained_Roadway_points


C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Project ID 827 is in the layer Constrained_Roadway_lines


C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Project ID 827 is NOT in the layer Constrained_Roadway_points
Project ID 828 is in the layer Constrained_Roadway_lines


...

Project ID 784 is in the layer Constrained_Roadway_lines


C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Project ID 784 is NOT in the layer Constrained_Roadway_points
Project ID 521 is in the layer Constrained_BikePed


C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Project ID 521 is NOT in the layer Constrained_BikePed_points
Project ID 812 is NOT in the layer Illustrative_BikePed


In [43]:
# remove duplicated spatial features
for layer in targetLayers:
    gdf = gpd.read_file(os.path.join(newPath, layer+'.shp'))
    if gdf.shape[0] > len(gdf.RTP_ID.unique()):
        IDs = [item for item, count in collections.Counter(gdf.RTP_ID.values).items() if count > 1]
        Counts = [count for item, count in collections.Counter(gdf.RTP_ID.values).items() if count > 1]
        print("ID {0} are duplicated {1} times in layer {2}".format(IDs, Counts, layer))
        gdf.drop_duplicates(inplace=True, ignore_index=True)
    else:
        print("{0} has all unique IDs".format(layer))  

ID [333, 924, 918, 828] are duplicated [2, 2, 2, 2] times in layer Constrained_Roadway_lines
ID [828, 924] are duplicated [2, 2] times in layer Constrained_Roadway_points
ID [66] are duplicated [2] times in layer Illustrative_Roadway_lines
Illustrative_Roadway_points has all unique IDs
ID [813] are duplicated [4] times in layer Constrained_BikePed
ID [721, 720] are duplicated [2, 5] times in layer Constrained_BikePed_points
Illustrative_BikePed has all unique IDs


In [11]:
data = RTP.checkDiff()

In [12]:
data.columns

Index(['Name40', 'GeographicLimits40', 'Description40',
       'PrimaryJurisdiction40', 'EstimatedCost40',
       'EstimatedYearofConstruction40', 'YearofConstructionCostMin40',
       'YearofConstructionCostMax40', 'Length40', 'RTP40', 'Category40',
       'Name45', 'GeographicLimits45', 'Description45',
       'PrimaryJurisdiction45', 'EstimatedCost45',
       'EstimatedYearofConstruction45', 'YearofConstructionCostMin45',
       'YearofConstructionCostMax45', 'Length45', 'RTP45', 'Category45',
       'NameDiff', 'GeographicLimitsDiff', 'DescriptionDiff',
       'PrimaryJurisdictionDiff', 'EstimatedCostDiff',
       'EstimatedYearofConstructionDiff', 'YearofConstructionCostMinDiff',
       'YearofConstructionCostMaxDiff', 'LengthDiff', 'RTPDiff',
       'CategoryDiff'],
      dtype='object')

In [13]:
data['GeographicLimitsDiff'].value_counts()

0    139
Name: GeographicLimitsDiff, dtype: int64

In [14]:
data['CategoryDiff'].value_counts()

0    139
Name: CategoryDiff, dtype: int64

In [15]:
data['LengthDiff'].value_counts()

0.0    138
Name: LengthDiff, dtype: int64

In [17]:
(df45.shape[0] - data.shape[0])/df45.shape[0]

0.4908424908424908

In [7]:
dfex = RTP.modifyRTP(RTP.combineTables(excludeTransit = True))

In [10]:
dfex.shape

(298, 13)

In [11]:
df45ex = RTP.modifyRTP(RTP.combineTables(year=2045, excludeTransit = True))

In [9]:
df45ex.shape

(254, 12)

In [2]:
dataex = RTP.checkDiff(excludeTransit = True, by='RTP')

In [3]:
dataex.columns

Index(['Name40', 'GeographicLimits40', 'Description40',
       'PrimaryJurisdiction40', 'EstimatedYearofConstruction40', 'Length40',
       'RTP', 'JurisdictionalProject#40', 'Comments40', 'Category40', 'Name45',
       'GeographicLimits45', 'Description45', 'PrimaryJurisdiction45',
       'EstimatedYearofConstruction45', 'Length45', 'JurisdictionalProject#45',
       'Category45', 'NameDiff', 'GeographicLimitsDiff', 'DescriptionDiff',
       'PrimaryJurisdictionDiff', 'EstimatedYearofConstructionDiff',
       'LengthDiff', 'JurisdictionalProject#Diff', 'CategoryDiff'],
      dtype='object')

In [4]:
dataex.shape

(176, 26)

In [12]:
df45ex.shape[0] - dataex.shape[0]

78

In [5]:
dataex.head()

,Name40,GeographicLimits40,Description40,PrimaryJurisdiction40,EstimatedYearofConstruction40,Length40,RTP,JurisdictionalProject#40,Comments40,Category40,...,JurisdictionalProject#45,Category45,NameDiff,GeographicLimitsDiff,DescriptionDiff,PrimaryJurisdictionDiff,EstimatedYearofConstructionDiff,LengthDiff,JurisdictionalProject#Diff,CategoryDiff
0,Delta/ Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,2020-2024,0.25,638,NaN,NaN,Added Freeway Lanes or Major Interchange Impro...,...,ETSP MM-3 (part),Added Freeway Lanes or Major Interchange Impro...,999,NaN,999,0,0,0.0,999.0,0
1,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Randy Pape Belltine Highway and p...,"ODOT, Eugene",2030-2034,1.1,312,MM-20,Updated - Rob,Added Freeway Lanes or Major Interchange Impro...,...,ETSP MM-20,Added Freeway Lanes or Major Interchange Impro...,0,0.0,999,0,0,0.0,999.0,0
2,Main Street,@ 48th Street,Traffic control improvements,Springfield,2020-2024,0.0,69,NaN,change cost from $290k to $300k,Arterial Capacity Improvements,...,NaN,Arterial Capacity Improvements,0,999.0,0,0,999,0.0,NaN,0
3,Main Street,@ Mountaingate Drive,Traffic control improvements - signal,Springfield,2020-2024,0.0,75,R-48,change cost from $290k to $900k,Arterial Capacity Improvements,...,R-48,Arterial Capacity Improvements,0,999.0,0,0,999,0.0,0.0,0
4,42nd Street,@ Marcola Road,Roundabout,Springfield,2025-2029,0.0,712,R-32,change to roundabout project and change cost f...,Arterial Capacity Improvements,...,R-32,Arterial Capacity Improvements,0,999.0,0,0,999,0.0,0.0,0


In [7]:
dataex['NameDiff'].value_counts()

0      158
999     18
Name: NameDiff, dtype: int64

In [13]:
dataex['GeographicLimitsDiff'].value_counts()

0.0      140
999.0     34
Name: GeographicLimitsDiff, dtype: int64

In [15]:
dataex['LengthDiff'].value_counts()

 0.00    174
 0.07      1
-0.70      1
Name: LengthDiff, dtype: int64

In [18]:
dataex['CategoryDiff'].value_counts()

0    176
Name: CategoryDiff, dtype: int64

In [9]:
dataex[dataex.NameDiff == 999][['Name40', 'Description40', 'Name45', 'Description45', 'RTP']]

,Name40,Description40,Name45,Description45,RTP
0,Delta/ Beltline Interchange,Interim/safety improvements; replace/revise ex...,Delta/Beltline Interchange,Interim/safety improvements; replace/revise ex...,638
33,Glacier Drive,Construct new collector with 2-lane cross-section,New Collector,Construct new collector with 2-lane cross-sect...,22
56,Main St. and 52nd St./Hwy 126 Int.,Interchange Plans,OR126E (Expressway) at 52nd St and Main St,Interchange Area Management Plans,96
57,Eugene-Springfield Hwy.,Facility Plan,OR126 Expressway,Facility Plan,835
61,OR 126,Study a new crossing of OR 126 between 5th Str...,Main Street (OR126B),Study a new crossing of OR 126 between 5th Str...,823
70,Main Street,Access plan study,Main Street (OR126B),Facility Plan,917
75,South 14th Street,Extend South 14th Street south of the Union Pa...,South 14th Street Extension,Extend South 14th Street south of the Union Pa...,825
76,New Collector,Extend South B Street with a 3-lane cross-sect...,New Collector - South B Street,Extend South B Street with a 3-lane cross-sect...,913
86,Eugene-Springfield Highway (SR-126),Interchange improvements,Eugene-Springfield Highway (OR-126E) at Pionee...,Interchange improvements,727
87,Eugene-Springfield Highway (SR-126),Widen to 6 lanes,Eugene-Springfield Highway (OR-126E),Widen to 6 lanes,728


In [14]:
dataex[dataex.GeographicLimitsDiff == 999][['GeographicLimits40', 'Description40', 'GeographicLimits45', 'Description45', 'RTP']]

,GeographicLimits40,Description40,GeographicLimits45,Description45,RTP
2,@ 48th Street,Traffic control improvements,at 48th Street,Traffic control improvements,69
3,@ Mountaingate Drive,Traffic control improvements - signal,at Mountaingate Drive,Traffic control improvements - signal,75
4,@ Marcola Road,Roundabout,at Marcola Road,Roundabout,712
5,@ Pheasant Boulevard,Traffic control improvements,at Pheasant Blvd,Traffic control improvements,744
6,@ Harlow Road,Traffic control improvements,at Harlow Road,Traffic control improvements,785
21,Bob Straub Parkway - Mountaingate Drive,Construct new 3-lane collector,Bob Straub Parkway to Mountaingate Drive and F...,Construct a new collector with a three-lane cr...,81
25,31st Street to 33rd Street,Extend existing street as 2-lane collector,31st Street to 35th Street,Construct Yolanda Avenue from 31st Street to 3...,783
28,"Game Farm Road East, to International Way",Construct new 3- lane collector,Game Farm Road East to International Way,Construct new 3- lane collector with sidewalks...,707
33,48th Street/Holly to South 55th Street,Construct new collector with 2-lane cross-section,Holly Street - South 48th Street to South 57th...,Construct new collector with 2-lane cross-sect...,22
42,Warren to UGB,Upgrade to urban facility,Warren St to Eugene UGB,Construct to eugene's minor arterial standards...,343


In [17]:
dataex[dataex.LengthDiff != 0][['Length40', 'Description40', 'Length45', 'Description45', 'RTP']]

,Length40,Description40,Length45,Description45,RTP
13,0.91,Add center turn lane.,0.21,Add center turn lane on Martin Luther King Jr....,602
14,0.07,Widen Barger Drive to provide a second through...,0.14,Widen Barger Drive to provide a second through...,497


In [5]:
dfex.head()

,Name,GeographicLimits,Description,PrimaryJurisdiction,EstimatedCost,EstimatedYearofConstruction,YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP,JurisdictionalProject#,Comments,Category
0,Eugene-Springfield Highway (SR-126),at Main Street,Construct interchange,ODOT,50000000.0,2030-2034,7.666397e+07,8.662156e+07,0.0,27,NaN,NaN,New Arterial Link or Interchange
1,Eugene-Springfield Highway (SR-126),at 52nd Street,Construct interchange,ODOT,40000000.0,2025-2029,5.264874e+07,5.948708e+07,0.0,30,NaN,NaN,New Arterial Link or Interchange
2,Beltline Local Arterial Bridge,Beaver Street to Delta Highway,Construct new arterial bridge with up to 4 lan...,"ODOT, Lane County, City of Eugene",83000000.0,2025-2029,1.092461e+08,1.234357e+08,0.95,512,ETSP MM-3,NaN,New Arterial Link or Interchange
3,Delta/ Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,20000000.0,2020-2024,2.259772e+07,2.553285e+07,0.25,638,NaN,NaN,Added Freeway Lanes or Major Interchange Impro...
4,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Randy Pape Belltine Highway and p...,"ODOT, Eugene",28100000.0,2030-2034,4.308515e+07,4.868131e+07,1.1,312,MM-20,Updated - Rob,Added Freeway Lanes or Major Interchange Impro...


In [22]:
df45ex.head()

,Name,GeographicLimits,Description,PrimaryJurisdiction,EstimatedCost,EstimatedYearofConstruction,YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP,JurisdictionalProject#,Category
0,Beltline Local Arterial Bridge,Beaver Street to Delta Highway,Construct new 2-lane arterial bridge over the ...,"ODOT, Lane County, City of Eugene",83000000.0,2025-2029,NaN,NaN,0.95,512,ETSP MM-3,New Arterial Link or Interchange
1,Eugene-Springfield Highway (SR-126),at Main Street,Construct interchange,ODOT,50000000.0,2030-2034,NaN,NaN,0.0,27,STSP R-43,New Arterial Link or Interchange
2,Eugene-Springfield Highway (SR-126),at 52nd Street,Construct interchange,ODOT,40000000.0,2025-2029,NaN,NaN,0.0,30,STSP R-40,New Arterial Link or Interchange
3,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Belltine Highway and provide inte...,"ODOT, Eugene",28100000.0,2030-2034,NaN,NaN,1.1,312,ETSP MM-20,Added Freeway Lanes or Major Interchange Impro...
4,Delta/Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,20000000.0,2020-2024,NaN,NaN,0.25,638,ETSP MM-3 (part),Added Freeway Lanes or Major Interchange Impro...


In [24]:
len(df45ex.RTP.unique())

218

In [33]:
df45ex_uni = df45ex[df45ex.columns.drop(["EstimatedCost", "YearofConstructionCostMin", "YearofConstructionCostMax"])]

In [37]:
df45ex_uni.drop_duplicates(subset='RTP', keep=False, inplace=True)

<ipython-input-37-aa41aa65dd80>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df45ex_uni.drop_duplicates(subset='RTP', keep=False, inplace=True)


In [38]:
df45ex_uni.shape

(188, 9)

In [40]:
len(df45ex_uni.RTP.unique())

188

In [39]:
df45ex_uni.head()

,Name,GeographicLimits,Description,PrimaryJurisdiction,EstimatedYearofConstruction,Length,RTP,JurisdictionalProject#,Category
3,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Belltine Highway and provide inte...,"ODOT, Eugene",2030-2034,1.1,312,ETSP MM-20,Added Freeway Lanes or Major Interchange Impro...
4,Delta/Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,2020-2024,0.25,638,ETSP MM-3 (part),Added Freeway Lanes or Major Interchange Impro...
6,Main Street,at 48th Street,Traffic control improvements,Springfield,2025-2029,0.0,69,NaN,Arterial Capacity Improvements
7,Main Street,at Mountaingate Drive,Traffic control improvements - signal,Springfield,2025-2029,0.0,75,R-48,Arterial Capacity Improvements
8,42nd Street,at Marcola Road,Roundabout,Springfield,2020-2024,0.0,712,R-32,Arterial Capacity Improvements


In [23]:
df45ex.shape

(254, 12)

In [2]:
data = RTP.getCombinedTables(cat='common')[0]

In [4]:
data.columns

Index(['Category40', 'Category45', 'CategoryDiff', 'Description40',
       'Description45', 'DescriptionDiff', 'EstimatedCost40',
       'EstimatedCost45', 'EstimatedCostDiff', 'EstimatedYearofConstruction40',
       'EstimatedYearofConstruction45', 'EstimatedYearofConstructionDiff',
       'GeographicLimits40', 'GeographicLimits45', 'GeographicLimitsDiff',
       'JurisdictionalProject#40', 'JurisdictionalProject#45',
       'JurisdictionalProject#Diff', 'Length40', 'Length45', 'LengthDiff',
       'Name40', 'Name45', 'NameDiff', 'PrimaryJurisdiction40',
       'PrimaryJurisdiction45', 'PrimaryJurisdictionDiff', 'RTP',
       'YearofConstructionCostMax40', 'YearofConstructionCostMax45',
       'YearofConstructionCostMaxDiff', 'YearofConstructionCostMin40',
       'YearofConstructionCostMin45', 'YearofConstructionCostMinDiff'],
      dtype='object')

In [13]:
data[data.GeographicLimitsDiff != 0][['RTP', 'GeographicLimits40', 'GeographicLimits45', 'Length40', 'Length45']].dropna()

,RTP,GeographicLimits40,GeographicLimits45,Length40,Length45
6,69,@ 48th Street,at 48th Street,0.0,0.0
7,75,@ Mountaingate Drive,at Mountaingate Drive,0.0,0.0
8,712,@ Marcola Road,at Marcola Road,0.0,0.0
9,744,@ Pheasant Boulevard,at Pheasant Blvd,0.0,0.0
10,785,@ Harlow Road,at Harlow Road,0.5,0.5
30,81,Bob Straub Parkway - Mountaingate Drive,Bob Straub Parkway to Mountaingate Drive and F...,1.03,1.03
34,783,31st Street to 33rd Street,31st Street to 35th Street,0.2,0.2
38,707,"Game Farm Road East, to International Way",Game Farm Road East to International Way,0.18,0.18
44,22,48th Street/Holly to South 55th Street,Holly Street - South 48th Street to South 57th...,0.94,0.94
55,343,Warren to UGB,Warren St to Eugene UGB,1.6,1.6


In [16]:
data[data.LengthDiff == 999 ][['RTP', 'Length40', 'Length45', 'GeographicLimits40', 'GeographicLimits45']]

,RTP,Length40,Length45,GeographicLimits40,GeographicLimits45
22,602,0.91,0.21,Leo Harris Parkway West to Centennial Loop,Leo Harris Parkway West to Centennial Loop
23,497,0.07,0.14,West of Primrose Street to where the street wi...,West of Primrose Street to where the street wi...


In [4]:
data2 = RTP.getCombinedTables(cat='common', export=True)[1]

In [5]:
data2.shape

(240, 12)

In [6]:
data2.columns

Index(['Category', 'Description', 'EstimatedCost',
       'EstimatedYearofConstruction', 'GeographicLimits',
       'JurisdictionalProject#', 'Length', 'Name', 'PrimaryJurisdiction',
       'YearofConstructionCostMax', 'YearofConstructionCostMin', 'RTP'],
      dtype='object')

In [7]:
data2.head()

,Category,Description,EstimatedCost,EstimatedYearofConstruction,GeographicLimits,JurisdictionalProject#,Length,Name,PrimaryJurisdiction,YearofConstructionCostMax,YearofConstructionCostMin,RTP
0,New Arterial Link or Interchange,Construct interchange,50000000.0,2030-2034,at Main Street,STSP R-43,0.0,Eugene-Springfield Highway (SR-126),ODOT,NaN,NaN,27
1,New Arterial Link or Interchange,Construct interchange,40000000.0,2025-2029,at 52nd Street,STSP R-40,0.0,Eugene-Springfield Highway (SR-126),ODOT,NaN,NaN,30
2,New Arterial Link or Interchange,Construct new 2-lane arterial bridge over the ...,83000000.0,2025-2029,Beaver Street to Delta Highway,ETSP MM-3,0.95,Beltline Local Arterial Bridge,"ODOT, Lane County, City of Eugene",NaN,NaN,512
3,Added Freeway Lanes or Major Interchange Impro...,Interim/safety improvements; replace/revise ex...,20000000.0,2020-2024,NaN,ETSP MM-3 (part),0.25,Delta/Beltline Interchange,ODOT,NaN,NaN,638
4,Added Freeway Lanes or Major Interchange Impro...,Add lanes on Belltine Highway and provide inte...,28100000.0,2030-2034,Roosevelt Boulevard to W. 11th Avenue,ETSP MM-20,1.1,Randy Pape Beltline Highway,"ODOT, Eugene",NaN,NaN,312
